In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

matplotlib: 3.3.4
seaborn   : 0.11.1
numpy     : 1.20.1
pandas    : 1.1.5



In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import PredefinedSplit

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df_type_1_features = pd.read_csv("../data/strains.dataset.tree.csv", index_col=0)

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (73,86,98,99,103,106,107,111,112,204,217,229,230,234,237,238,242,243) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_type_1_features['dataset_cat'] = pd.Series([_.split("_")[0] for _ in df_type_1_features['dataset']], dtype='category')
CV = PredefinedSplit(df_type_1_features['dataset_cat'].cat.codes)

In [6]:
for name, group in df_type_1_features.groupby('dataset'):
    print(len(group['assembly_accession'].unique()))

15921
18806
21665
21036
13506
16495
16662


In [7]:
df_type_1_features['truth'].sum() / df_type_1_features.shape[0]

0.0009831494628941664

In [8]:
list(df_type_1_features.columns)

['assembly_accession',
 'tree_closest_assembly_accession',
 'tree_dist',
 'tree_top_dist',
 'hits',
 'percent_coverage',
 'mean_coverage',
 'sd_coverage',
 'percent_padded_coverage',
 'mean_padded_coverage',
 'sd_padded_coverage',
 'percent_binned_coverage',
 'mean_binned_coverage',
 'sd_binned_coverage',
 'expected_percent_coverage',
 'shannon_entropy',
 'percent_max_uncovered_region',
 'largest_pileup',
 'largest_padded_pileup',
 'largest_binned_pileup',
 'gc_content',
 'total_genome_length',
 'ungapped_genome_length',
 'num_n_groups',
 'consecutive_ns',
 'gf_accession',
 'gf_ambiguous_bases',
 'gf_checkm_completeness',
 'gf_checkm_contamination',
 'gf_checkm_marker_count',
 'gf_checkm_marker_lineage',
 'gf_checkm_marker_set_count',
 'gf_checkm_strain_heterogeneity',
 'gf_coding_bases',
 'gf_coding_density',
 'gf_contig_count',
 'gf_gc_count',
 'gf_gc_percentage',
 'gf_genome_size',
 'gf_gtdb_genome_representative',
 'gf_gtdb_representative',
 'gf_gtdb_taxonomy',
 'gf_gtdb_type_desig

In [9]:
features = ['hits',
 'percent_coverage',
 'mean_coverage',
 'sd_coverage',
 'percent_binned_coverage',
 'mean_binned_coverage',
 'sd_binned_coverage',
 'expected_percent_coverage',
 'shannon_entropy',
 'percent_max_uncovered_region',
 'largest_pileup',
 'largest_binned_pileup',
 'gc_content',
 'total_genome_length',
 'ungapped_genome_length',
 'num_n_groups',
 'consecutive_ns',
 'tree_dist',
 'tree_top_dist',
 'gf_checkm_completeness',
 'gf_checkm_contamination'
]

In [10]:
X_type_1 = df_type_1_features[features + ["assembly_accession", "dataset", "truth", "dataset_cat"]]
                                                 
X_type_1 = X_type_1.replace([np.inf, -np.inf], np.nan)
X_type_1 = X_type_1.dropna()

print(X_type_1.groupby('dataset').sum())

y = X_type_1["truth"]
X = X_type_1.loc[:, X_type_1.columns.difference(["assembly_accession", "dataset", "dataset_cat", "truth"])]

X_type_1.to_csv("../data/type_1.extra.tree.csv", index=False)

                    hits  percent_coverage  mean_coverage    sd_coverage  \
dataset                                                                    
dual_index       1734304         28.062051      88.341678     840.304259   
gis_20         361319935         40.965084   11045.607897  214833.055608   
hmp_even        26120868         48.138912     588.211176   14515.805304   
hmp_staggered   13987473         23.541141     275.052218    9609.165257   
mbarc_26        89680807         32.067000    6017.729981   16506.313587   
zymo_even       10317373         19.231281     485.308514    5010.760035   
zymo_log       133887236         13.417510    4209.986755   65399.745693   

               percent_binned_coverage  mean_binned_coverage  \
dataset                                                        
dual_index                     45.7004              173.4298   
gis_20                         69.4328            36131.9216   
hmp_even                      100.7977             2612.079

In [11]:
from sklearn.feature_selection import RFECV

from sklearn.linear_model import LogisticRegression

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
from tpot import TPOTClassifier


?TPOTClassifier

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/tpot/builtins/one_hot_encoder.py:216: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, categorical_features='auto', dtype=np.float,


Init signature:
TPOTClassifier(
    generations=100,
    population_size=100,
    offspring_size=None,
    mutation_rate=0.9,
    crossover_rate=0.1,
    scoring=None,
    cv=5,
    subsample=1.0,
    n_jobs=1,
    max_time_mins=None,
    max_eval_time_mins=5,
    random_state=None,
    config_dict=None,
    template=None,
    warm_start=False,
    memory=None,
    use_dask=False,
    periodic_checkpoint_folder=None,
    early_stop=None,
    verbosity=0,
    disable_update_check=False,
    log_file=None,
)
Docstring:      TPOT estimator for classification problems.
Init docstring:
Set up the genetic programming algorithm for pipeline optimization.

Parameters
----------
generations: int or None, optional (default: 100)
    Number of iterations to the run pipeline optimization process.
    It must be a positive number or None. If None, the parameter
    max_time_mins must be defined as the runtime limit.
    Generally, TPOT will work better when you give it more generations (and
    the

In [13]:
from tpot.config.classifier import classifier_config_dict

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
# 'class_weight': 'balanced'

In [26]:
classifier_config_dict = {

    # Classifiers
    'sklearn.naive_bayes.GaussianNB': {
    },

    'sklearn.naive_bayes.BernoulliNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    },

    'sklearn.naive_bayes.MultinomialNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    },

    'sklearn.tree.DecisionTreeClassifier': {
        'criterion': ["gini", "entropy"],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21)
    },

    'sklearn.ensemble.ExtraTreesClassifier': {
        'n_estimators': [100],
        'criterion': ["gini", "entropy"],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'bootstrap': [True, False],
        'class_weight': ['balanced', 'balanced_subsample', None]
    },

    'sklearn.ensemble.RandomForestClassifier': {
        'n_estimators': [100],
        'criterion': ["gini", "entropy"],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf':  range(1, 21),
        'bootstrap': [True, False],
        'class_weight': ['balanced', 'balanced_subsample', None]
    },

    'sklearn.ensemble.GradientBoostingClassifier': {
        'n_estimators': [100],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'max_features': np.arange(0.05, 1.01, 0.05)
    },

    'sklearn.neighbors.KNeighborsClassifier': {
        'n_neighbors': range(1, 101),
        'weights': ["uniform", "distance"],
        'p': [1, 2]
    },

    'sklearn.svm.LinearSVC': {
        'penalty': ["l1", "l2"],
        'loss': ["hinge", "squared_hinge"],
        'dual': [True, False],
        'tol': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
        'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.]
    },

    'sklearn.linear_model.LogisticRegression': {
        'penalty': ["l1", "l2"],
        'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.],
        'dual': [True, False],
        'class_weight': ['balanced', None]
    },

    'xgboost.XGBClassifier': {
        'n_estimators': [100],
        'max_depth': range(1, 11),
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'subsample': np.arange(0.05, 1.01, 0.05),
        'min_child_weight': range(1, 21),
        'n_jobs': [1],
        'verbosity': [0],
        'scale_pos_weight': [1, 10, 25, 50, 75, 99, 100, 1000]
    },

    'sklearn.linear_model.SGDClassifier': {
        'loss': ['log', 'hinge', 'modified_huber', 'squared_hinge', 'perceptron'],
        'penalty': ['elasticnet'],
        'alpha': [0.0, 0.01, 0.001],
        'learning_rate': ['invscaling', 'constant'],
        'fit_intercept': [True, False],
        'l1_ratio': [0.25, 0.0, 1.0, 0.75, 0.5],
        'eta0': [0.1, 1.0, 0.01],
        'power_t': [0.5, 0.0, 1.0, 0.1, 100.0, 10.0, 50.0]
    },

    'sklearn.neural_network.MLPClassifier': {
        'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
        'learning_rate_init': [1e-3, 1e-2, 1e-1, 0.5, 1.]
    },

    # Preprocesssors
    'sklearn.preprocessing.Binarizer': {
        'threshold': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.decomposition.FastICA': {
        'tol': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.cluster.FeatureAgglomeration': {
        'linkage': ['ward', 'complete', 'average'],
        'affinity': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
    },

    'sklearn.preprocessing.MaxAbsScaler': {
    },

    'sklearn.preprocessing.MinMaxScaler': {
    },

    'sklearn.preprocessing.Normalizer': {
        'norm': ['l1', 'l2', 'max']
    },

    'sklearn.kernel_approximation.Nystroem': {
        'kernel': ['rbf', 'cosine', 'chi2', 'laplacian', 'polynomial', 'poly', 'linear', 'additive_chi2', 'sigmoid'],
        'gamma': np.arange(0.0, 1.01, 0.05),
        'n_components': range(1, 11)
    },

    'sklearn.decomposition.PCA': {
        'svd_solver': ['randomized'],
        'iterated_power': range(1, 11)
    },

    'sklearn.preprocessing.PolynomialFeatures': {
        'degree': [2],
        'include_bias': [False],
        'interaction_only': [False]
    },

    'sklearn.kernel_approximation.RBFSampler': {
        'gamma': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.preprocessing.RobustScaler': {
    },

    'sklearn.preprocessing.StandardScaler': {
    },

    'tpot.builtins.ZeroCount': {
    },

    'tpot.builtins.OneHotEncoder': {
        'minimum_fraction': [0.05, 0.1, 0.15, 0.2, 0.25],
        'sparse': [False],
        'threshold': [10]
    },

    # Selectors
    'sklearn.feature_selection.SelectFwe': {
        'alpha': np.arange(0, 0.05, 0.001),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.SelectPercentile': {
        'percentile': range(1, 100),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.VarianceThreshold': {
        'threshold': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    },

    'sklearn.feature_selection.RFE': {
        'step': np.arange(0.05, 1.01, 0.05),
        'estimator': {
            'sklearn.ensemble.ExtraTreesClassifier': {
                'n_estimators': [100],
                'criterion': ['gini', 'entropy'],
                'max_features': np.arange(0.05, 1.01, 0.05),
                'class_weight': ['balanced', 'balanced_subsample', None]
            }
        }
    },

    'sklearn.feature_selection.SelectFromModel': {
        'threshold': np.arange(0, 1.01, 0.05),
        'estimator': {
            'sklearn.ensemble.ExtraTreesClassifier': {
                'n_estimators': [100],
                'criterion': ['gini', 'entropy'],
                'max_features': np.arange(0.05, 1.01, 0.05),
                'class_weight': ['balanced', 'balanced_subsample', None]
            }
        }
    }

}

0.7601246105919003


In [ ]:
from tpot import TPOTClassifier


tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2, random_state=42, n_jobs=40, config_dict=classifier_config_dict, cv=CV, scoring='f1')
tpot.fit(X, y)
# print(tpot.score(X_test, y_test))
tpot.export('tpot_iris_pipeline.py')

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from tpot.export_utils import set_param_recursive

# Average CV score on the training set was: 0.4020165729147765
exported_pipeline = make_pipeline(
    MaxAbsScaler(),
    RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample", criterion="gini", max_features=0.6000000000000001, min_samples_leaf=9, min_samples_split=9, n_estimators=100)
)

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
from sklearn.metrics import f1_score

In [88]:
classifiers = []
results = []
for i, (train, test) in enumerate(CV.split(X, y)):
    clf = RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample", criterion="gini", max_features=0.6000000000000001, min_samples_leaf=9, min_samples_split=9, n_estimators=100)
    clf.fit(X.loc[train], y.loc[train])
    classifiers.append(clf)
    y_pred = clf.predict(X.loc[test])
    results.append(f1_score(y.loc[test], y_pred))
    print(results)

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.41025641025641024]
[0.41025641025641024, 0.39999999999999997]
[0.41025641025641024, 0.39999999999999997, 0.23999999999999996]
[0.41025641025641024, 0.39999999999999997, 0.23999999999999996, 0.2777777777777778]
[0.41025641025641024, 0.39999999999999997, 0.23999999999999996, 0.2777777777777778, 0.4864864864864864]


In [62]:
f = np.mean(np.array(results))

In [63]:
q = (np.sum(y) / y.shape[0])

In [73]:
y.sum()

122

In [81]:
f

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.3588082416743853

In [86]:
clf.oob_score_

/home/bhillmann/.conda/envs/tpot-cuml/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'Pipeline' object has no attribute 'oob_score_'

In [84]:
f1_score(y, clf.predict(X))

0.6948640483383687

In [64]:
(f - q) / (1 - q)

0.3581772339666864